In [37]:
import numpy as np
import string
import re
import csv
from collections import Counter
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
import itertools
import operator
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix

train = np.load("./data/data_train.pkl",allow_pickle=True)
test=np.load("./data/data_test.pkl",allow_pickle=True)
X_train=train[0]
y_train=train[1]


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nithi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

In [39]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(X_train)
Train_X_Tfidf = Tfidf_vect.fit_transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(test)

In [18]:
from sklearn.naive_bayes import ComplementNB

C_navie = ComplementNB(alpha=2.5)
C_navie.fit(Train_X_Tfidf,y_train)
predictions_C_navie=C_navie.predict(Test_X_Tfidf)
print("C Bayes Accuracy Score -> ",accuracy_score(predictions_C_navie, y_test)*100)

C Bayes Accuracy Score ->  58.3


In [25]:
from sklearn.linear_model import LogisticRegression

lg=LogisticRegression(C=3 ,solver='sag',multi_class='multinomial',max_iter=500)
lg.fit(Train_X_Tfidf, y_train)

LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
y_pred2 = lg.predict(Test_X_Tfidf)
print('accuracy %s' % accuracy_score(y_pred2, y_test))

accuracy 0.5542857142857143


In [40]:
from sklearn.svm import LinearSVC 
from sklearn.ensemble import VotingClassifier

clf1 = ComplementNB()
clf2 = LinearSVC(loss='hinge', penalty='l2', C= 1, max_iter= 4000, multi_class= 'ovr' )
clf3 = MultinomialNB()
clf4=LogisticRegression(C=3 ,solver='sag',multi_class='multinomial',max_iter=500)

eclf = VotingClassifier(estimators= [('CNB', clf1), ('LSVC', clf2), ('MNB', clf3)], voting= 'hard')
eclf.fit(Train_X_Tfidf, y_train)

VotingClassifier(estimators=[('CNB',
                              ComplementNB(alpha=1.0, class_prior=None,
                                           fit_prior=True, norm=False)),
                             ('LSVC',
                              LinearSVC(C=1, class_weight=None, dual=True,
                                        fit_intercept=True, intercept_scaling=1,
                                        loss='hinge', max_iter=4000,
                                        multi_class='ovr', penalty='l2',
                                        random_state=None, tol=0.0001,
                                        verbose=0)),
                             ('MNB',
                              MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True))],
                 flatten_transform=True, n_jobs=None, voting='hard',
                 weights=None)

In [41]:
new_pred = eclf.predict(Test_X_Tfidf)

#print('accuracy %s' % accuracy_score(new_pred, y_test))

In [43]:
import csv
csvfile=open("submission123.csv",'w', newline='')
obj=csv.writer(csvfile)
obj.writerow(("Id","Category"))
for rowIndex,pred in enumerate(new_pred):
    #test=Test.predict(test_sample)
    #print(rowIndex)
    #pred_class=Test.unique_class_Names[test]
    obj.writerow((rowIndex,pred))
csvfile.close()